In [ ]:
import pandas as pd
import numpy as np 
import os

In [ ]:
#db_dir = '/home/panastas/mount_karol/PhD_Research/PARALiA-GEMMex/karol_build/karol_8A100-install/Database/'
db_dir = '/home/panastas/mount_karol/PhD_Research/PARALiA-GEMMex/karol_build/karol_8A100-install/Database/'

db_dir_mb = db_dir + 'microbenchmarks'
CHL_WORKERS = 8
CHL_MEMLOCS = CHL_WORKERS + 2
memloc_list = []
for i in range(0,CHL_MEMLOCS):
    memloc_list.append(i)

In [ ]:
def translate_binary_to_unit_list(case_id):
    id_list = []
    for mask_offset in range(0, CHL_WORKERS+1):
        mask = 1 << mask_offset;
        if (case_id & mask):
            id_list.append(mask_offset);
    return id_list

def translate_unit_list_to_binary(list_input):
    case_id_out = 0; 
    for elem in list_input:
        case_id_out+= pow(2, elem);
    return case_id_out

def binary_case_id_split(case_id):
    active_unit_id_list = translate_binary_to_unit_list(case_id)
    if (len(active_unit_id_list)%2 != 0):
        return None
    out_active_unit_id_list = []
    for idx in range(0,len(active_unit_id_list),2):
        #print (idx)
        out_active_unit_id_list.append(active_unit_id_list[idx])
    return translate_unit_list_to_binary(out_active_unit_id_list)

def binary_case_id_slide_right(case_id, slides):
    active_unit_id_list = translate_binary_to_unit_list(case_id)
    out_active_unit_id_list = []
    for idx in range(0,len(active_unit_id_list)):
        out_active_unit_id_list.append(active_unit_id_list[idx] + slides)
        #print(active_unit_id_list[idx] + slides)
    return translate_unit_list_to_binary(out_active_unit_id_list);

def bench_bc_log_read(filename):
    if os.path.exists(filename):
        temp = pd.read_csv(filename, header=None, names = ['dest','dim1', 'dim2', 'elemSize','bw'], 
            dtype = {'dest': np.int64, 'dim1': np.int64, 'dim2': np.int64, 'elemSize': np.int64, 'bw': np.float64} )
        read_flag = 1
    else:
        temp = pd.DataFrame()
        read_flag = 0
    return read_flag, temp

def bench_bc_grid_log_read(filename):
    if os.path.exists(filename):
        temp = pd.read_csv(filename, header=None, names = ['dest', 'src', 'dim1', 'dim2', 'elemSize','bw'], 
            dtype = {'dest': np.int64, 'src': np.int64, 'dim1': np.int64, 'dim2': np.int64, 'elemSize': np.int64, 'bw': np.float64} )
        read_flag = 1
    else:
        temp = pd.DataFrame()
        read_flag = 0
    return read_flag, temp

def bench_bc_groupby_(df):
    result = df.groupby(['T','dev_num', 'dev_id_token', 'cachesize_limit','TransA','TransB',
        'alpha','beta','M','N','K','Asloc','Bsloc','Csloc', 'CsOutloc', 'flops', 'bytes', 'loc_str', 'imp', 'plot_order'], as_index=False, sort=True).mean()
    #print(result)
    return result

In [ ]:
if False:
    for case_id in range(1,2**CHL_WORKERS):
        for case_id_rev in [case_id,binary_case_id_split(case_id)]:
            if case_id_rev == None:
                continue
            bench_bc = pd.DataFrame()
            cont_flag = 0
            for elem in memloc_list:
                bw_bench_sc_filename =  '%s/bw_bench_scatter_2D_%d_%d.log' %(db_dir_mb, elem, case_id)
                read_flag, bench_sc_el = bench_bc_log_read(bw_bench_sc_filename)
                if read_flag == 0:
                    cont_flag = 1
                    break
                #print(bench_sc_el.to_markdown())
                bench_sc_el['src'] = elem
                bench_bc = pd.concat([bench_bc,bench_sc_el])
            if cont_flag:
                continue
            cont_flag = 0
            for elem in memloc_list:
                bw_bench_gc_filename =  '%s/bw_bench_gather_2D_%d_%d.log' %(db_dir_mb, elem, case_id_rev)
                read_flag, bench_gc_el = bench_bc_log_read(bw_bench_gc_filename)
                if read_flag == 0:
                    cont_flag = 1
                    break
                #print(bench_gc_el.to_markdown())
                bench_gc_el.rename(columns={"dest": "src"}, inplace = True)
                bench_gc_el['dest'] = elem
                bench_bc = pd.concat([bench_bc,bench_gc_el])
            if cont_flag:
                continue
            print(case_id)
            active_unit_id_list = translate_binary_to_unit_list(case_id)
            print(active_unit_id_list)
            active_rev_unit_id_list = translate_binary_to_unit_list(case_id_rev)
            print(active_rev_unit_id_list)
            #bench_bc.sort_values(inplace = True, by = ['dim1', 'dest', 'src'])
            #print(bench_bc.to_markdown()) 
            max_dim = bench_bc['dim1'].max()
            #print(max_dim)
            bench_bc_gb = bench_bc[bench_bc['dim1'] == max_dim]
            #print(bench_bc_gb.to_markdown())
            bench_grid = pd.DataFrame()
            cont_flag = 0
            for elem in range(CHL_WORKERS+1,CHL_MEMLOCS):
                #print(elem)
                bw_bench_gr_filename =  '%s/bw_bench_bidirectional_grid_2D_%d_%d_%d_%d.log' %(db_dir_mb, elem, case_id, case_id, case_id_rev)
                read_flag, bench_gr_el = bench_bc_grid_log_read(bw_bench_gr_filename)
                if read_flag == 0:
                    cont_flag = 1
                    break
                bench_grid = pd.concat([bench_grid,bench_gr_el])
            if cont_flag:
                continue
            max_dim = bench_grid['dim1'].max()
            #print(max_dim)
            bench_grid = bench_grid[bench_grid['dim1'] == max_dim]
            print('====================================================================================================')
            print('CHL_MEMLOCS = %d\n' %(CHL_MEMLOCS))
            print('One-directional:')
            for dest in memloc_list:
                #print('%d |' %(elem), end = '')
                for src in memloc_list:
                    bws = bench_bc_gb[bench_bc_gb['src'] == src]
                    bwd = bws[bws['dest'] == dest]['bw']
                    if(bwd.empty == False):
                        print('%.3lf ' %(float(bwd.max())), end = '')
                    else:
                        print('-1.000 ', end = '')
                print('')
            print('\nBidirectional:')
            for dest in memloc_list:
                #print('%d |' %(elem), end = '')
                for src in memloc_list:
                    bws = bench_grid[bench_grid['src'] == src]
                    bwd = bws[bws['dest'] == dest]['bw']
                    if(bwd.empty == False):
                        print('%.3lf ' %(float(bwd.max())), end = '')
                    else:
                        print('-1.000 ', end = '')
                print('')
            print('====================================================================================================')

In [ ]:
bench_bc = pd.DataFrame()
cont_flag = 0
for elem in range(0,CHL_WORKERS):
    bw_bench_scatter_filename =  '%s/bw_bench_scatter_2D_%d_%d.log' %(db_dir_mb, elem, 2**CHL_WORKERS - 1)
    sc_read_flag, bench_sc_el = bench_bc_log_read(bw_bench_scatter_filename)
    bw_bench_gather_filename =  '%s/bw_bench_gather_2D_%d_%d.log' %(db_dir_mb, elem, 2**CHL_WORKERS - 1)
    gt_read_flag, bench_gt_el = bench_bc_log_read(bw_bench_gather_filename)
    if sc_read_flag == 0 or gt_read_flag == 0: 
        cont_flag = 1
        break
    #print(bench_sc_el.to_markdown())
    bench_sc_el['src'] = elem
    bench_gt_el.rename(columns={"dest": "src"}, inplace = True)
    bench_gt_el['dest'] = elem
    bench_bc = pd.concat([bench_bc,bench_sc_el, bench_gt_el])

for case_id in range(1,2**CHL_WORKERS):
    case_id_rev_list = [case_id]
    case_id_emulator_list = [case_id]
    half_split = binary_case_id_split(case_id)
    if half_split != None:
        case_id_rev_list.append(half_split)
        case_id_emulator_list.append(half_split)
        #for slide_right in range (1, CHL_WORKERS):
        #    slide_split = binary_case_id_slide_right(half_split, slide_right)
        #    if slide_split < (2**CHL_WORKERS):
        #        case_id_rev_list.append(slide_split)
        #        case_id_emulator_list.append(half_split)
    bench_bc_adjusted = bench_bc
    bench_grid = pd.DataFrame()
    cont_flag = 0
    ctr = 0
    for case_id_rev in case_id_rev_list:
        for elem in range(CHL_WORKERS,CHL_MEMLOCS):
            bw_bench_scatter_filename =  '%s/bw_bench_scatter_2D_%d_%d.log' %(db_dir_mb, elem, case_id)
            sc_read_flag, bench_sc_el = bench_bc_log_read(bw_bench_scatter_filename)
            bw_bench_gather_filename =  '%s/bw_bench_gather_2D_%d_%d.log' %(db_dir_mb, elem, case_id_rev)
            gt_read_flag, bench_gt_el = bench_bc_log_read(bw_bench_gather_filename)
            if sc_read_flag == 0 or gt_read_flag == 0: 
                cont_flag = 1
                break
            #print(bench_sc_el.to_markdown())
            bench_sc_el['src'] = elem
            bench_gt_el.rename(columns={"dest": "src"}, inplace = True)
            bench_gt_el['dest'] = elem
            bench_bc_adjusted = pd.concat([bench_bc_adjusted,bench_sc_el, bench_gt_el])
        for elem in range(CHL_WORKERS,CHL_MEMLOCS):
            bw_bench_gr_filename =  '%s/bw_bench_bidirectional_grid_2D_%d_%d_%d_%d.log' %(db_dir_mb, elem, case_id, case_id, case_id_rev)
            read_flag, bench_gr_el = bench_bc_grid_log_read(bw_bench_gr_filename)
            if read_flag == 0:
                cont_flag = 1
                break
            else:
                print(bw_bench_gr_filename)
            bench_grid = pd.concat([bench_grid,bench_gr_el])
        if cont_flag:
            continue
        #cont_flag = 0
        #print("Case IDs: Devices = %d, Queues = %d, Rev-queues = %s (emulating %s)" %(case_id, case_id, case_id_rev_list, case_id_emulator_list))
        active_unit_id_list = translate_binary_to_unit_list(case_id)
        active_unit_id_in = translate_binary_to_unit_list(case_id)
        active_rev_unit_id_list = translate_binary_to_unit_list(case_id_rev_list[ctr])
        active_rev_emul_id_list = translate_binary_to_unit_list(case_id_emulator_list[ctr])
        print("Devices = %s, Queues = %s, Rev-queues = %s (emulating %s)" %(active_unit_id_list, active_unit_id_in, active_rev_unit_id_list, active_rev_emul_id_list))
        ctr+= 1
        print('====================================================================================================')
        print('CHL_MEMLOCS = %d\n' %(CHL_MEMLOCS))
        print('One-directional:')
        for dest in memloc_list:
            #print('%d |' %(elem), end = '')
            for src in memloc_list:
                bwd = bench_bc_adjusted.loc[(bench_bc_adjusted['src'] == src) & (bench_bc_adjusted['dest'] == dest)]['bw']
                if(bwd.empty == False):
                    print('%.3lf ' %(float(bwd.max())), end = '')
                else:
                    print('-1.000 ', end = '')
            print('')
        print('\nBidirectional:')
        for dest in memloc_list:
            #print('%d |' %(elem), end = '')
            for src in memloc_list:
                bwd = bench_grid.loc[(bench_grid['src'] == src) & (bench_grid['dest'] == dest)]['bw']
                if(bwd.empty == False):
                    print('%.3lf ' %(float(bwd.max())), end = '')
                else:
                    print('-1.000 ', end = '')
            print('')
        print('====================================================================================================')